In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"]  = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
import findspark
findspark.init()

In [0]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DataCleaning_analysis').getOrCreate()

In [0]:
df = spark.read.csv('/content/drive/My Drive/Datasets/LoanStats_2018Q4.csv',inferSchema = "True",header = "True")

In [0]:
#df.printSchema()

In [224]:
df.count()

128416

In [0]:
df_sel = df.select(['term','home_ownership','grade','purpose',
                    'int_rate','addr_state','loan_status','application_type',
                    'loan_amnt','emp_length','annual_inc','dti','delinq_2yrs',
                    'revol_util','total_acc','num_tl_90g_dpd_24m','dti_joint'
                    ])

In [0]:
#df_sel.describe().show()

In [226]:
df_sel.describe(['term','loan_amnt','emp_length','annual_inc','dti','delinq_2yrs','revol_util','total_acc']).show()

+-------+----------+------------------+----------+------------------+------------------+-------------------+----------+------------------+
|summary|      term|         loan_amnt|emp_length|        annual_inc|               dti|        delinq_2yrs|revol_util|         total_acc|
+-------+----------+------------------+----------+------------------+------------------+-------------------+----------+------------------+
|  count|    128412|            128412|    128412|            128412|            128175|             128412|    128256|            128412|
|   mean|      null| 15971.32102139987|      null|  82797.3278609476| 19.93317753071978|0.22783696227766875|      null|22.677413325857398|
| stddev|      null|10150.384232741852|      null|108298.46579150103|20.143542243475455| 0.7337929617806055|      null|12.129215673024776|
|    min| 36 months|              1000|    1 year|               0.0|               0.0|                  0|        0%|                 2|
|    max| 60 months|       

In [227]:
df_sel.cache                                                                    #cache the dataset in executer cache memory.

<bound method DataFrame.cache of DataFrame[term: string, home_ownership: string, grade: string, purpose: string, int_rate: string, addr_state: string, loan_status: string, application_type: string, loan_amnt: int, emp_length: string, annual_inc: double, dti: double, delinq_2yrs: int, revol_util: string, total_acc: int, num_tl_90g_dpd_24m: int, dti_joint: double]>

In [180]:
df_sel.describe(['loan_amnt','emp_length','dti','delinq_2yrs','revol_util','total_acc']).show()

+-------+------------------+----------+------------------+-------------------+----------+------------------+
|summary|         loan_amnt|emp_length|               dti|        delinq_2yrs|revol_util|         total_acc|
+-------+------------------+----------+------------------+-------------------+----------+------------------+
|  count|            128412|    128412|            128175|             128412|    128256|            128412|
|   mean| 15971.32102139987|      null| 19.93317753071978|0.22783696227766875|      null|22.677413325857398|
| stddev|10150.384232741852|      null|20.143542243475455| 0.7337929617806055|      null|12.129215673024776|
|    min|              1000|    1 year|               0.0|                  0|        0%|                 2|
|    max|             40000|       n/a|             999.0|                 24|    99.90%|               160|
+-------+------------------+----------+------------------+-------------------+----------+------------------+



In [181]:
df.createOrReplaceTempView('loanStats')         #Temp table
spark.sql('select emp_length from loanstats limit 10').show()

+----------+
|emp_length|
+----------+
|  < 1 year|
|       n/a|
|   3 years|
| 10+ years|
| 10+ years|
|   6 years|
|   5 years|
| 10+ years|
| 10+ years|
|  < 1 year|
+----------+



In [0]:
from pyspark.sql.functions import regexp_extract,regexp_replace
from pyspark.sql.functions import col

In [228]:
regex_string = '\\d+'
df_sel = df_sel.withColumn("term_cleaned",regexp_replace(col('term'),'months','')).withColumn("emp_length_Cleaned",regexp_extract(col('emp_length'),regex_string,0))
df_sel.select('term','term_cleaned','emp_length','emp_length_Cleaned').show(10)

+----------+------------+----------+------------------+
|      term|term_cleaned|emp_length|emp_length_Cleaned|
+----------+------------+----------+------------------+
| 36 months|         36 |  < 1 year|                 1|
| 36 months|         36 |       n/a|                  |
| 36 months|         36 |   3 years|                 3|
| 36 months|         36 | 10+ years|                10|
| 60 months|         60 | 10+ years|                10|
| 36 months|         36 |   6 years|                 6|
| 60 months|         60 |   5 years|                 5|
| 36 months|         36 | 10+ years|                10|
| 60 months|         60 | 10+ years|                10|
| 60 months|         60 |  < 1 year|                 1|
+----------+------------+----------+------------------+
only showing top 10 rows



In [229]:
df_sel.printSchema()

root
 |-- term: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- application_type: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- num_tl_90g_dpd_24m: integer (nullable = true)
 |-- dti_joint: double (nullable = true)
 |-- term_cleaned: string (nullable = true)
 |-- emp_length_Cleaned: string (nullable = true)



In [230]:
df_sel.stat.crosstab('loan_status','grade').show()

+------------------+-----+-----+-----+-----+----+---+---+----+
| loan_status_grade|    A|    B|    C|    D|   E|  F|  G|null|
+------------------+-----+-----+-----+-----+----+---+---+----+
|   In Grace Period|   58|   98|  138|  111|  54|  8|  3|   0|
|        Fully Paid| 1436| 1557| 1365|  913| 415| 39| 12|   0|
|              null|    0|    0|    0|    0|   0|  0|  0|   4|
|           Default|    0|    3|    3|    4|   5|  0|  0|   0|
|Late (31-120 days)|   83|  195|  297|  262| 169| 18|  6|   0|
| Late (16-30 days)|   46|   82|  112|  100|  57|  2|  2|   0|
|           Current|36365|33874|29044|15640|5239|315| 73|   0|
|       Charged Off|   23|   52|   59|   42|  23|  5|  5|   0|
+------------------+-----+-----+-----+-----+----+---+---+----+



In [231]:
df_sel.createOrReplaceTempView('loanstats_sel')
spark.sql('select purpose,count(*) as count from loanstats_sel group by purpose order by count(*) desc').show()

+------------------+-----+
|           purpose|count|
+------------------+-----+
|debt_consolidation|70603|
|       credit_card|34961|
|  home_improvement| 7512|
|             other| 7094|
|    major_purchase| 2303|
|           medical| 1499|
|    small_business| 1051|
|               car| 1037|
|             house|  823|
|          vacation|  802|
|            moving|  656|
|  renewable_energy|   71|
|              null|    4|
+------------------+-----+



In [232]:
from pyspark.sql.functions import *
df_sel.select([ count(  when(isnan(c) | col(c).isNull(),c)).alias(c) for c in df_sel.columns]).show()

+----+--------------+-----+-------+--------+----------+-----------+----------------+---------+----------+----------+---+-----------+----------+---------+------------------+---------+------------+------------------+
|term|home_ownership|grade|purpose|int_rate|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc|dti|delinq_2yrs|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emp_length_Cleaned|
+----+--------------+-----+-------+--------+----------+-----------+----------------+---------+----------+----------+---+-----------+----------+---------+------------------+---------+------------+------------------+
|   4|             4|    4|      4|       4|         4|          4|               4|        4|         4|         4|241|          4|       160|        4|                 4|   111634|           4|                 4|
+----+--------------+-----+-------+--------+----------+-----------+----------------+---------+----------+----------+---+-----------+--------

In [0]:
df_sel = df_sel.na.drop('any',None,"loan_status")

In [234]:
df_sel.select([ count(  when(isnan(c) | col(c).isNull(),c)).alias(c) for c in df_sel.columns]).show()

+----+--------------+-----+-------+--------+----------+-----------+----------------+---------+----------+----------+---+-----------+----------+---------+------------------+---------+------------+------------------+
|term|home_ownership|grade|purpose|int_rate|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc|dti|delinq_2yrs|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emp_length_Cleaned|
+----+--------------+-----+-------+--------+----------+-----------+----------------+---------+----------+----------+---+-----------+----------+---------+------------------+---------+------------+------------------+
|   0|             0|    0|      0|       0|         0|          0|               0|        0|         0|         0|237|          0|       156|        0|                 0|   111630|           0|                 0|
+----+--------------+-----+-------+--------+----------+-----------+----------------+---------+----------+----------+---+-----------+--------

In [235]:
df_sel.describe(['dti_joint','revol_util']).show()

+-------+------------------+----------+
|summary|         dti_joint|revol_util|
+-------+------------------+----------+
|  count|             16782|    128256|
|   mean|19.226602312000985|      null|
| stddev|  8.14163126459558|      null|
|    min|               0.0|        0%|
|    max|             39.99|    99.90%|
+-------+------------------+----------+



In [0]:
df_sel = df_sel.withColumn('revol_util_cleaned',regexp_replace(col('revol_util'),"%",""))

In [0]:
df_sel.createOrReplaceTempView('loanstats_sel')

In [0]:
mean_revol_util = spark.sql('select avg(revol_util_cleaned) from loanstats_sel').collect()[0][0]
df_sel = df_sel.withColumn('mean_revol_util',lit(mean_revol_util))
df_sel = df_sel.withColumn('revol_util_filled',coalesce(col('revol_util_cleaned'),col('mean_revol_util')))

In [241]:
df_sel.select([ count(  when(isnan(c) | col(c).isNull(),c)).alias(c) for c in df_sel.columns]).show()

+----+--------------+-----+-------+--------+----------+-----------+----------------+---------+----------+----------+---+-----------+----------+---------+------------------+---------+------------+------------------+------------------+---------------+-----------------+
|term|home_ownership|grade|purpose|int_rate|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc|dti|delinq_2yrs|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emp_length_Cleaned|revol_util_cleaned|mean_revol_util|revol_util_filled|
+----+--------------+-----+-------+--------+----------+-----------+----------------+---------+----------+----------+---+-----------+----------+---------+------------------+---------+------------+------------------+------------------+---------------+-----------------+
|   0|             0|    0|      0|       0|         0|          0|               0|        0|         0|         0|237|          0|       156|        0|                 0|   111630|           0| 

In [0]:
df_sel.createOrReplaceTempView('loanstats_sel')

In [0]:
df_sel = spark.sql('select *, cast(revol_util_filled as double) as revol_util_final from loanstats_sel')

In [244]:
df_sel.printSchema()

root
 |-- term: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- application_type: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- num_tl_90g_dpd_24m: integer (nullable = true)
 |-- dti_joint: double (nullable = true)
 |-- term_cleaned: string (nullable = true)
 |-- emp_length_Cleaned: string (nullable = true)
 |-- revol_util_cleaned: string (nullable = true)
 |-- mean_revol_util: double (nullable = false)
 |-- revol_util_filled: string (nullable = false)
 |-- revol_util_final: double (nullable =

In [245]:
df_sel.describe(['revol_util','revol_util_final']).show()

+-------+----------+-----------------+
|summary|revol_util| revol_util_final|
+-------+----------+-----------------+
|  count|    128256|           128412|
|   mean|      null|44.19728121881271|
| stddev|      null|24.78377355680623|
|    min|        0%|              0.0|
|    max|    99.90%|            183.8|
+-------+----------+-----------------+



In [0]:
df_sel = df_sel.withColumn('final_dti',coalesce(col('dti'),col('dti_joint')))   #merging DTI and DTI_Joint

In [248]:
df_sel.select([ count(  when(isnan(c) | col(c).isNull(),c)).alias(c) for c in df_sel.columns]).show()

+----+--------------+-----+-------+--------+----------+-----------+----------------+---------+----------+----------+---+-----------+----------+---------+------------------+---------+------------+------------------+------------------+---------------+-----------------+----------------+---------+
|term|home_ownership|grade|purpose|int_rate|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc|dti|delinq_2yrs|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emp_length_Cleaned|revol_util_cleaned|mean_revol_util|revol_util_filled|revol_util_final|final_dti|
+----+--------------+-----+-------+--------+----------+-----------+----------------+---------+----------+----------+---+-----------+----------+---------+------------------+---------+------------+------------------+------------------+---------------+-----------------+----------------+---------+
|   0|             0|    0|      0|       0|         0|          0|               0|        0|         0|         0

In [254]:
df_sel.createOrReplaceTempView('loanstats_sel')
spark.sql('select loan_status , count(*) as count from loanstats_sel group by loan_status').show()

+------------------+------+
|       loan_status| count|
+------------------+------+
|        Fully Paid|  5737|
|           Default|    15|
|   In Grace Period|   470|
|       Charged Off|   209|
|Late (31-120 days)|  1030|
|           Current|120550|
| Late (16-30 days)|   401|
+------------------+------+



In [0]:
# merging all late type of loan_status into one column
# segregating load_status into two parts i.e. bad loan = yes/no

In [0]:
df_sel = df_sel.withColumn('bad_loan',when(col('loan_status').isin(['In Grace Period','Charged Off','Late (31-120 days)','Late (16-30 days)']),'yes').otherwise('no'))

In [261]:
df_sel.createOrReplaceTempView('loanstats_sel')
spark.sql('select bad_loan , count(*) as count from loanstats_sel group by bad_loan').show()

+--------+------+
|bad_loan| count|
+--------+------+
|      no|126302|
|     yes|  2110|
+--------+------+



In [265]:
df_sel.printSchema()

root
 |-- term: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- application_type: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- num_tl_90g_dpd_24m: integer (nullable = true)
 |-- dti_joint: double (nullable = true)
 |-- term_cleaned: string (nullable = true)
 |-- emp_length_Cleaned: string (nullable = true)
 |-- revol_util_cleaned: string (nullable = true)
 |-- mean_revol_util: double (nullable = false)
 |-- revol_util_filled: string (nullable = false)
 |-- revol_util_final: double (nullable =

In [0]:
final_df = df_sel.drop('term','emp_length','dti','dti_joint','revol_util_filled','revol_util_cleaned','mean_revol_util')

In [270]:
final_df.printSchema()

root
 |-- home_ownership: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- application_type: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- num_tl_90g_dpd_24m: integer (nullable = true)
 |-- term_cleaned: string (nullable = true)
 |-- emp_length_Cleaned: string (nullable = true)
 |-- revol_util_final: double (nullable = true)
 |-- final_dti: double (nullable = true)
 |-- bad_loan: string (nullable = false)



In [272]:
final_df.stat.crosstab('bad_loan','grade').show()         # as we movie forward in grades the ratio of bad loan increases

+--------------+-----+-----+-----+-----+----+---+---+
|bad_loan_grade|    A|    B|    C|    D|   E|  F|  G|
+--------------+-----+-----+-----+-----+----+---+---+
|           yes|  210|  427|  606|  515| 303| 33| 16|
|            no|37801|35434|30412|16557|5659|354| 85|
+--------------+-----+-----+-----+-----+----+---+---+



##creating a permanent table !



In [0]:
final_df.write.format('parquet').saveAsTable('LC_Loan_Data')

In [281]:
spark.sql('select * from lc_loan_Data').show(5)

+--------------+-----+------------------+--------+----------+-----------+----------------+---------+----------+-----------+----------+---------+------------------+------------+------------------+----------------+---------+--------+
|home_ownership|grade|           purpose|int_rate|addr_state|loan_status|application_type|loan_amnt|annual_inc|delinq_2yrs|revol_util|total_acc|num_tl_90g_dpd_24m|term_cleaned|emp_length_Cleaned|revol_util_final|final_dti|bad_loan|
+--------------+-----+------------------+--------+----------+-----------+----------------+---------+----------+-----------+----------+---------+------------------+------------+------------------+----------------+---------+--------+
|      MORTGAGE|    B|debt_consolidation|  10.33%|        OR|    Current|      Individual|    10000|  280000.0|          2|       38%|       23|                 0|         36 |                 1|            38.0|     6.15|      no|
|      MORTGAGE|    B|  home_improvement|  12.98%|        KY|    Current